In [1]:
from manim import *
config.media_embed = True

In [132]:
%%manim -v WARNING --format=mp4 --quality=h -r 1920,1080 BalancinGIF
from manim import *
import numpy as np
from scipy.integrate import solve_ivp

class BalancinGIF(Scene):
    def construct(self):
        # Configuración de fondo blanco
        self.camera.background_color = WHITE
        
        # Parámetros y constantes
        m1, m2, m_b = 0.25, 0.352, 0.5
        L, g, b, u_c, eps, F = 1, 9.81, 0.04, 0.002, 1e-4, 0 

        # Escalado visual mejorado
        escala_sistema = 4
        offset_medicion = 0.6  

        # Cálculos dinámicos
        J = L**2 * ((1/12)*m_b + (1/4)*(m1 + m2))
        alpha = b / J
        beta = (u_c * g * L) / (2 * J)
        gamma = ((m1 - m2) * g * L) / (2 * J)
        delta = L / (2 * J)

        # Condición inicial (importante para la animación)
        theta_inicial = 1
        theta_p_inicial = 3

        # Solución numérica con más puntos
        max_tiempo = 30  # Duración total en segundos
        t_eval = np.linspace(0, max_tiempo, 1800)  # 60 frames por segundo
        sol = solve_ivp(lambda t, x: [
            x[1],
            -alpha*x[1] - beta*np.tanh(x[1]/eps) + gamma*np.cos(x[0]) + delta*F
        ], [0, max_tiempo], [theta_inicial, theta_p_inicial], t_eval=t_eval)
        theta, theta_p = sol.y

        # Configuración gráfica del balancín
        L_visual = L * escala_sistema
        radio_base = 1
        radio_m1 = m1 * radio_base
        radio_m2 = m2 * radio_base

        # Elementos gráficos con bordes negros - Ahora el pivote está en el centro (ORIGEN)
        pivot = Dot(ORIGIN, color=BLACK, radius=0.12)
        beam = Line(LEFT*L_visual, RIGHT*L_visual, color=BLUE_D, stroke_width=escala_sistema)
        mass1 = Circle(radius=radio_m1, color=RED_D, fill_opacity=1, stroke_width=3).move_to(LEFT*L_visual/2)
        mass2 = Circle(radius=radio_m2, color=GREEN_D, fill_opacity=1, stroke_width=3).move_to(RIGHT*L_visual/2)

        # Elementos de medición
        medicion_style = {"stroke_width":3, "color":GREY_BROWN, "dash_length":0.15}
        distancia_line = DashedLine(LEFT*L_visual/4, RIGHT*L_visual/4, **medicion_style)
        L_label = Tex("L", font_size=32, color=GREY_BROWN)

        # Anotaciones estáticas
        friccion = Tex(r"$b, \mu_c$", color=BLACK, font_size=24).next_to(pivot, DOWN, buff=0.1)
        
        # Etiquetas de masa
        etiqueta_m1 = Tex(r"$m_1$", color=BLACK).scale(0.8)
        etiqueta_m2 = Tex(r"$m_2$", color=BLACK).scale(0.8)

        # Configuración del espacio de fase - Ajustado para mejor visualización
        margin = 0.5
        fase_axes = Axes(
            x_range=[min(theta)-margin, max(theta)+margin, margin],  # Rango más simétrico
            y_range=[min(theta_p)-margin, max(theta_p)+margin, margin],    # Rango más amplio
            x_length=5,
            y_length=5,
            axis_config={"color": BLACK, "include_ticks": True, "font_size": 20, "stroke_width":3},
            tips=True
        ).to_edge(RIGHT, buff=0.8)
        
        fase_labels = VGroup(
            MathTex(r"\theta \, [\text{rad}]", color=BLACK, font_size=24)
            .next_to(fase_axes.x_axis, RIGHT, buff=0.1),
            
            MathTex(r"\dot{\theta} [\text{rad/s}]", color=BLACK, font_size=24)
            .next_to(fase_axes.y_axis, UP, buff=0.1)
        )

        # Elementos animados del espacio de fase
        fase_line = VMobject(color=RED, stroke_width=3)
        fase_dot = Dot(color=BLUE, radius=0.1, fill_opacity=0.8)
        
        # Punto de condición inicial (punto verde)
        condicion_inicial_dot = Dot(
            fase_axes.coords_to_point(theta_inicial, theta_p_inicial), 
            color=GREEN, 
            radius=0.12
        )
        
        # Etiqueta para la condición inicial
        condicion_inicial_label = Tex(
            f"\n$\\theta_0 = {theta_inicial}$ rad\n$\\dot{{\\theta}}_0 = {theta_p_inicial}$ rad/s", 
            color=BLACK, 
            font_size=20
        ).next_to(condicion_inicial_dot, LEFT, buff=0.6)
        
        # Líneas de referencia para la condición inicial
        h_line = DashedLine(
            fase_axes.coords_to_point(theta_inicial, 0),
            fase_axes.coords_to_point(theta_inicial, theta_p_inicial),
            color=BLACK, 
            stroke_width=3
        )
        
        v_line = DashedLine(
            fase_axes.coords_to_point(0, theta_p_inicial),
            fase_axes.coords_to_point(theta_inicial, theta_p_inicial),
            color=BLACK, 
            stroke_width=3
        )
        
        # Título en LaTeX
        fase_title = Tex(
            r"Espacio Fase: $\theta$ vs $\dot{\theta}$", 
            color=BLACK, 
            font_size=28
        ).next_to(fase_axes, UP, buff=0.4)

        # Grupo principal para organización - Balancín movido más a la izquierda
        balancin_group = VGroup(pivot, beam, mass1, mass2, distancia_line, L_label, friccion).shift(LEFT*4)
        fase_group = VGroup(
            fase_axes, 
            fase_labels, 
            fase_title, 
            fase_line, 
            fase_dot,
            condicion_inicial_dot,
            condicion_inicial_label,
            h_line,
            v_line
        )

        self.add(balancin_group, fase_group)

        # Animación mejorada
        tiempo = ValueTracker(0)
        
        def actualizar_sistema(mob):
            t = tiempo.get_value()
            # CORRECCIÓN: Interpolación al rango completo de tiempo (30 segundos)
            indice = int(np.interp(t, [0, max_tiempo], [0, len(theta)-1]))
            angulo = theta[indice]
            velocidad = theta_p[indice]
            
            # Vector perpendicular para las etiquetas
            normal = np.array([-np.sin(angulo), np.cos(angulo), 0])
            
            # Actualizar posición de la barra y masas
            pivot_pos = balancin_group[0].get_center()
            x1 = L_visual/2*np.cos(angulo)
            y1 = L_visual/2*np.sin(angulo)
            beam.become(Line(
                pivot_pos + np.array([-x1, -y1, 0]), 
                pivot_pos + np.array([x1, y1, 0]), 
                color=BLUE_D, 
                stroke_width=escala_sistema
            ))
            
            mass1_pos = pivot_pos + np.array([-x1, -y1, 0])
            mass2_pos = pivot_pos + np.array([x1, y1, 0])
            
            mass1.move_to(mass1_pos)
            mass2.move_to(mass2_pos)
            etiqueta_m1.move_to(mass1.get_center())
            etiqueta_m2.move_to(mass2.get_center())

            # Actualizar medición
            start_point = mass1_pos + normal * offset_medicion/2
            end_point = mass2_pos + normal * offset_medicion/2
            distancia_line.become(DashedLine(start_point, end_point, **medicion_style))
            L_label.move_to((start_point + end_point)/2)

            # Añadir etiquetas dinámicas al grupo
            self.add(etiqueta_m1, etiqueta_m2, L_label)

            # Actualizar espacio de fase
            if indice > 0:
                current_points = np.array([
                    fase_axes.coords_to_point(theta[i], theta_p[i])
                    for i in range(indice+1)
                ])
                
                fase_line.set_points_smoothly(current_points)
            fase_dot.move_to(fase_axes.coords_to_point(theta[indice], theta_p[indice]))

        beam.add_updater(actualizar_sistema)
        self.play(
            tiempo.animate.set_value(max_tiempo),
            run_time=max_tiempo,
            rate_func=linear
        )

Manim Community v0.18.1